# Clustering Toronto boroughs

In [2]:
import numpy as np
import pandas as pd
import json
import requests 
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)


from bs4 import BeautifulSoup
#from pandas.io.json import json_normalize
#from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim
#!pip install folium
#import folium
!pip install geocoder
import geocoder

print('Done!')

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

## Scrapping data from Wikipedia with requests & BS4 (not assigned replaced)

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page=requests.get(url,'lxml').text
soup=BeautifulSoup(wiki_page,'lxml')

header=['Postcode', 'Borough', 'Neighbourhood']
core=[]

tabl=soup.find('table')
for tr in tabl.find_all('tr'):
    td=tr.find_all('td')
    if len(td)==3:
      instance=['','','']
      instance[0]=td[0].text.strip()
      instance[1]=td[1].text.strip()
      if td[2].text.strip().replace('\n','')=='Not assigned':
           instance[2]=td[1].text.strip()
      else:
           instance[2]=td[2].text.strip().replace('\n','') 
      
      core.append(instance)
    
df=pd.DataFrame.from_records(core, columns=header)

df1=df[df.Borough != 'Not assigned']
df1.sort_values(by=['Postcode'])
df1.reset_index(drop=True, inplace=True)
df1.head()



,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Merging rows with same postcodes

In [4]:
df2=df1.copy()
i=0
df2.is_copy=False
for i in range(df2.shape[0]):
    if i != df2.shape[0]-1:
        if df2.loc[i,'Postcode']==df2.loc[i+1,'Postcode']:
            df2.loc[i+1,'Borough']='Duplicate'
            df2.loc[i,'Neighbourhood']=df2.loc[i,'Neighbourhood']+', '+df2.loc[i+1,'Neighbourhood']
            for e in range(i+1,df2.shape[0]):
                if e != df2.shape[0]-1: 
                    if df2.loc[e,'Postcode']==df2.loc[e+1,'Postcode']:
                        df2.loc[e+1,'Borough']='Duplicate'
                        df2.loc[i,'Neighbourhood']=df2.loc[i,'Neighbourhood']+', '+df2.loc[e+1,'Neighbourhood']
                    else:
                        i=e+1
                        break
df3=df2[df2.Borough != 'Duplicate']   
df3.reset_index(drop=True,inplace=True)
df3.shape

(103, 3)

## Retrieveng coordinates with Bing Maps API

In [ ]:
# The code was removed by Watson Studio for sharing.

In [5]:
def longlat(adress):
        g = geocoder.bing('Toronto, Ontario' + adress, key=bing_credentials)
        lat_lng_coords = g.latlng
        return lat_lng_coords
df4=df3.copy()

df4['Latitude']=0
df4['Longitude']=0

for i in range (df4.shape[0]):
    
    df4.loc[i,'Longitude']=longlat(df4.loc[i,'Postcode'])[1]
    df4.loc[i,'Latitude']=longlat(df4.loc[i,'Postcode'])[0]
    #print('Toronto, Ontario ',df4.loc[i,'Postcode'],longlat(df4.loc[i,'Postcode'])[1],longlat(df4.loc[i,'Postcode'])[0])
   
df4.head(15)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751255,-79.329895
1,M4A,North York,Victoria Village,43.729958,-79.314201
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655220,-79.361969
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722801,-79.450691
4,M7A,Queen's Park,Queen's Park,43.664486,-79.393021
5,M9A,Etobicoke,Islington Avenue,43.662743,-79.528427
6,M1B,Scarborough,"Rouge, Malvern",43.810154,-79.194603
7,M3B,North York,Don Mills North,43.749134,-79.362007
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.707577,-79.310913
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657467,-79.377708
